# Grid Distillation Different Dataset Demos

In [12]:
import os

import numpy as np
import matplotlib.pyplot as plt


from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import grid_distillation as gd



In [10]:
"""class GridImageClusterSampler(ImageClusterSampler):
    def plot_clusters_with_grids(self, n_samples):
        plt.figure(figsize=(10, 6))
        for i in range(self.n_clusters):
            cluster_data = self.X_pca[self.cluster_labels == i]
            plt.scatter(cluster_data[:, 0], cluster_data[:, 1], label=f'Cluster {i}', alpha=0.5)
        
        grid_size = int(np.ceil(np.sqrt(n_samples)))
        x_min, x_max = np.min(self.X_pca[:, 0]), np.max(self.X_pca[:, 0])
        y_min, y_max = np.min(self.X_pca[:, 1]), np.max(self.X_pca[:, 1])
        
        x_grid = np.linspace(x_min, x_max, grid_size + 1)
        y_grid = np.linspace(y_min, y_max, grid_size + 1)
        
        for x in x_grid:
            plt.axvline(x=x, color='k', linestyle='--', alpha=0.5)
        for y in y_grid:
            plt.axhline(y=y, color='k', linestyle='--', alpha=0.5)
        
        plt.title('Clustering of Cats and Dogs with Grids')
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plt.legend()
        plt.show()
    
    def get_grid_sampled_indices(self, cluster_indices, n_samples):
        if len(cluster_indices) <= n_samples:
            return cluster_indices
        
        grid_size = int(np.ceil(np.sqrt(n_samples)))
        x_min, x_max = np.min(self.X_pca[:, 0]), np.max(self.X_pca[:, 0])
        y_min, y_max = np.min(self.X_pca[:, 1]), np.max(self.X_pca[:, 1])
        
        x_grid = np.linspace(x_min, x_max, grid_size + 1)
        y_grid = np.linspace(y_min, y_max, grid_size + 1)
        
        sampled_indices = []
        for i in range(grid_size):
            for j in range(grid_size):
                x_start, x_end = x_grid[i], x_grid[i + 1]
                y_start, y_end = y_grid[j], y_grid[j + 1]
                
                grid_indices = [
                    idx for idx in cluster_indices 
                    if x_start <= self.X_pca[idx, 0] < x_end and y_start <= self.X_pca[idx, 1] < y_end
                ]
                
                if grid_indices:
                    sampled_indices.append(np.random.choice(grid_indices))

        while len(sampled_indices) < n_samples:
            sampled_indices.append(np.random.choice(cluster_indices))
        
        return sampled_indices[:n_samples]

    def get_selected_samples(self, n_samples):
        selected_samples = []
        for label in range(2):
            label_indices = np.where(self.y == label)[0]
            label_cluster_labels = self.cluster_labels[label_indices]
            unique_clusters = np.unique(label_cluster_labels)
            
            for cluster_id in unique_clusters:
                cluster_indices = label_indices[label_cluster_labels == cluster_id]
                selected_indices = self.get_grid_sampled_indices(cluster_indices, n_samples)
                selected_samples.extend(selected_indices)
        return selected_samples"""

"class GridImageClusterSampler(ImageClusterSampler):\n    def plot_clusters_with_grids(self, n_samples):\n        plt.figure(figsize=(10, 6))\n        for i in range(self.n_clusters):\n            cluster_data = self.X_pca[self.cluster_labels == i]\n            plt.scatter(cluster_data[:, 0], cluster_data[:, 1], label=f'Cluster {i}', alpha=0.5)\n        \n        grid_size = int(np.ceil(np.sqrt(n_samples)))\n        x_min, x_max = np.min(self.X_pca[:, 0]), np.max(self.X_pca[:, 0])\n        y_min, y_max = np.min(self.X_pca[:, 1]), np.max(self.X_pca[:, 1])\n        \n        x_grid = np.linspace(x_min, x_max, grid_size + 1)\n        y_grid = np.linspace(y_min, y_max, grid_size + 1)\n        \n        for x in x_grid:\n            plt.axvline(x=x, color='k', linestyle='--', alpha=0.5)\n        for y in y_grid:\n            plt.axhline(y=y, color='k', linestyle='--', alpha=0.5)\n        \n        plt.title('Clustering of Cats and Dogs with Grids')\n        plt.xlabel('PCA Component 1')\n  

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Load the dataset
digits = load_digits()
X = digits.images
y = digits.target

# Normalize the images
X = X / 255.0

# Reshape the images to (n_samples, 1, 8, 8)
X = X.reshape(-1, 1, 8, 8)

# Convert labels to torch tensors
y = torch.tensor(y, dtype=torch.long)

# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), y_train)
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 2 * 2, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 2 * 2)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Train and evaluate the original model
train(model, train_loader, criterion, optimizer, epochs=10)
original_accuracy = evaluate(model, test_loader)
print(f'Original Model Accuracy: {original_accuracy}%')

# distill the dataset by grid
n_samples = 100
grid = gd.GridImageClusterSampler(X_train, y_train, n_clusters=10)
grid.plot_clusters_with_grids(100)

# get the selected samples
selected_samples = grid.get_selected_samples(100)

# get the distilled dataset
X_distilled = X_train[selected_samples]
y_distilled = y_train[selected_samples]
# Convert distilled dataset to torch tensors
X_distilled = torch.tensor(X_distilled, dtype=torch.float32)
y_distilled = torch.tensor(y_distilled, dtype=torch.long)

# Create a DataLoader for the distilled dataset
distilled_dataset = TensorDataset(X_distilled, y_distilled)
distilled_loader = DataLoader(distilled_dataset, batch_size=32, shuffle=True)

# Initialize a new model for the distilled dataset
model_distilled = SimpleCNN()
optimizer_distilled = optim.Adam(model_distilled.parameters(), lr=0.001)

# Train and evaluate the distilled model
train(model_distilled, distilled_loader, criterion, optimizer_distilled, epochs=10)
distilled_accuracy = evaluate(model_distilled, test_loader)
print(f'Distilled Model Accuracy: {distilled_accuracy}%')

# Compare the performance of the two models
print(f'Original Model Accuracy: {original_accuracy}%')
print(f'Distilled Model Accuracy: {distilled_accuracy}%')


Epoch [1/10], Loss: 2.3069718890719946
Epoch [2/10], Loss: 2.303570016225179
Epoch [3/10], Loss: 2.3020217312706843
Epoch [4/10], Loss: 2.2982140329149034
Epoch [5/10], Loss: 2.2692815992567272
Epoch [6/10], Loss: 2.102637102868822
Epoch [7/10], Loss: 1.7324073049757216
Epoch [8/10], Loss: 1.4329083972507053
Epoch [9/10], Loss: 1.2389584064483643
Epoch [10/10], Loss: 1.0651254759894477
Original Model Accuracy: 74.72222222222223%


TypeError: 'NoneType' object is not subscriptable

<Figure size 1000x600 with 0 Axes>

In [11]:
# load a dataset(sklearn laod digits) of images and labels from a directory, 
# train a pytorch model on it, distill the dataset (grid) and train a new model on the distilled dataset
# and compare the performance of the two models

# load the dataset
digits = load_digits()
X = digits.images

# reshape the images to 3D
X = X.reshape(-1, 8, 8, 1)

# normalize the images
X = X / 255

# one hot encode the labels
y = digits.target
y = np.eye(10)[y]

# split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a model in torch

model = torch.nn.Sequential(


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# distill the dataset
# create a grid
grid = gd.GridImageClusterSampler(X_train, y_train, n_clusters=10, n_samples=10)
grid.plot_clusters_with_grids(10)

# get the selected samples
selected_samples = grid.get_selected_samples(100)

# get the distilled dataset
X_distilled = X_train[selected_samples]
y_distilled = y_train[selected_samples]

# use the same model architecture
model_distilled = Sequential()
model_distilled.add(Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 1)))
model_distilled.add(MaxPooling2D((2, 2)))
model_distilled.add(Conv2D(64, (3, 3), activation='relu'))
model_distilled.add(MaxPooling2D((2, 2)))
model_distilled.add(Conv2D(64, (3, 3), activation='relu'))
model_distilled.add(Flatten())
model_distilled.add(Dense(64, activation='relu'))
model_distilled.add(Dense(10, activation='softmax'))

model_distilled.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the distilled model
model_distilled.fit(X_distilled, y_distilled, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# compare the performance of the two models
model.evaluate(X_test, y_test)
model_distilled.evaluate(X_test, y_test)


NameError: name 'Sequential' is not defined